In [1]:
import arcpy
import os
from arcpy import env

In [2]:
# Setting up the environment
env.workspace = "D:/AALTO UNIVERSITY/B.SC ECON - IMMIGRATION & PUBLIC TRANSPORTATION/IMMIGRANT PATTERN & PUBLIC TRANSPORT ACCESSIBILITY/DATA"

# Define paths
central_folder = os.path.join(env.workspace, "Travel_Time_Matrix_tables")
ykr_grid_table = os.path.join(env.workspace, "MetropAccess_YKR_grid", "MetropAccess_YKR_grid_EurefFIN.shp")
district_table = os.path.join(env.workspace, "HMA_districts", "HMA_districts.shp")
merged_ykr_table = os.path.join(env.workspace, "merged_ykr_table.shp")

In [3]:
file_list = [os.path.join(central_folder, f) for f in os.listdir(central_folder) if f.endswith('.txt')]
print(len(file_list))

45


In [4]:
# The environment settings
arcpy.env.overwriteOutput = True
arcpy.env.workspace = env.workspace

# Create a temporary geodatabase
merged_gdb = "merged.gdb"
arcpy.CreateFileGDB_management(env.workspace, merged_gdb)

<Result 'D:/AALTO UNIVERSITY/B.SC ECON - IMMIGRATION & PUBLIC TRANSPORTATION/IMMIGRANT PATTERN & PUBLIC TRANSPORT ACCESSIBILITY/Data\\merged.gdb'>

In [5]:
# Convert the shapefile to a geodatabase feature class
merged_ykr_table_gdb = os.path.join(env.workspace, merged_gdb, "merged_ykr_table_gdb")
arcpy.FeatureClassToFeatureClass_conversion(ykr_grid_table, os.path.join(env.workspace, merged_gdb), "merged_ykr_table_gdb")

for file in file_list:
    year_id = os.path.basename(file).replace(".txt", "")
    year, file_id = year_id.split('-')
    in_table = file
    temp_table_path = os.path.join(env.workspace, merged_gdb, f"temp_table_{year}_{file_id}")

    # Convert the text file to a table in the geodatabase
    arcpy.TableToTable_conversion(in_table, os.path.join(env.workspace, merged_gdb), f"temp_table_{year}_{file_id}", "", "", "SEMICOLON")

    # Rename desired columns in the geodatabase table
    desired_fields = ["pt_m_tt", "pt_m_t", "pt_m_d"]
    for field_name in desired_fields:
        new_name = f"{field_name}_{year}_{file_id}"
        if len(new_name) > 60:
            new_name = new_name[:60]
        arcpy.AlterField_management(temp_table_path, field_name, new_name, new_name)

    # Delete unwanted fields from the table
    fields_to_delete = [f.name for f in arcpy.ListFields(temp_table_path)
                        if f.name not in [f"{f}_{year}_{file_id}" for f in desired_fields]
                        and f.name not in ["from_id", "to_id", "OID", "OBJECTID"]]
    if fields_to_delete:
        arcpy.DeleteField_management(temp_table_path, fields_to_delete)

    # Join with the merged YKR grid table in the geodatabase
    arcpy.JoinField_management(merged_ykr_table_gdb, "YKR_ID", temp_table_path, "from_id")

    # Delete unwanted fields `from_id` and `to_id`
    arcpy.DeleteField_management(merged_ykr_table_gdb, ["from_id", "to_id"])

    arcpy.Delete_management(temp_table_path)

# Optionally, convert the geodatabase feature class back to a shapefile
# arcpy.FeatureClassToShapefile_conversion([merged_ykr_table_gdb], env.workspace)

# Clean up: delete the temporary geodatabase
# arcpy.Delete_management(os.path.join(env.workspace, merged_gdb))

In [6]:
# 1. Transform all -1 values to NULL in the merged YKR table.
fields = arcpy.ListFields(merged_ykr_table_gdb)
for field in fields:
    if field.type in ['Double', 'Integer', 'Single', 'SmallInteger']:
        with arcpy.da.UpdateCursor(merged_ykr_table_gdb, [field.name]) as cursor:
            for row in cursor:
                if row[0] == -1:
                    row[0] = None
                cursor.updateRow(row)

In [7]:
def field_exists_in_field_mappings(field_name, field_mappings):
    """Check if a field exists in the field mappings."""
    for i in range(field_mappings.fieldCount):
        if field_mappings.getFieldMap(i).outputField.name == field_name:
            return True
    return False

# Prepare field mappings for the spatial join
field_mappings = arcpy.FieldMappings()

larger_district_file = os.path.join(env.workspace, merged_gdb, "joined_districts")

# List of reserved or system fields to exclude
reserved_fields = ["FID", "Shape", "OBJECTID", "Shape_Length", "Shape_Area"]

# Add all fields from the district_table to the field mappings
for field in arcpy.ListFields(district_table):
    if field.name not in reserved_fields and not field_exists_in_field_mappings(field.name, field_mappings):
        field_map = arcpy.FieldMap()
        field_map.addInputField(district_table, field.name)
        field_mappings.addFieldMap(field_map)

# List of fields to exclude from the YKR grid table
exclude_fields = ["OBJECTID", "Shape", "x", "y", "YKR_ID", "Shape_Length", "Shape_Area"]

for field in arcpy.ListFields(merged_ykr_table_gdb):
    if field.name not in exclude_fields and not field_exists_in_field_mappings(field.name, field_mappings):
        field_map = arcpy.FieldMap()
        field_map.addInputField(merged_ykr_table_gdb, field.name)
        field_map.mergeRule = "Min"
        field_mappings.addFieldMap(field_map)

# Perform the spatial join
arcpy.analysis.SpatialJoin(target_features=district_table, 
                           join_features=merged_ykr_table_gdb,
                           out_feature_class=larger_district_file, 
                           join_type="KEEP_COMMON", 
                           field_mapping=field_mappings, 
                           match_option="INTERSECT")


<Result 'D:\\AALTO UNIVERSITY\\B.SC ECON - IMMIGRATION & PUBLIC TRANSPORTATION\\IMMIGRANT PATTERN & PUBLIC TRANSPORT ACCESSIBILITY\\Data\\merged.gdb\\joined_districts'>